In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import re

In [ ]:
def normalize_text(text):
    # Keep only words, alphabets, and numbers
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

    return text

In [ ]:
def list_of_sentence(text):
    los = []
    sentences = sent_tokenize(text.lower())
    for sen in sentences:
        # sen = normalize_text(sen)
        tokensen = word_tokenize(sen)
        los.append(tokensen)
    return los

In [ ]:
def write_list_of_lists_to_text(file_path, data, delimiter='\t'):
    with open(file_path, 'w') as file:
        for sublist in data:
            line = delimiter.join(sublist)
            file.write(line + '\n')

def import_text_to_list_of_lists(file_path, delimiter='\t'):
    result = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespace and newlines
            sublist = line.split(delimiter)
            result.append(sublist)
    return result

In [ ]:
def page_contents(link, div_class = 'elementor-widget-container'):
    driver = webdriver.Safari()
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    div_elements = soup.find_all('div', class_=div_class)

    paragraphs = []
    for div in div_elements:
        div_paragraphs = div.find_all('p')
        for paragraph in div_paragraphs:
            paragraphs.append(paragraph.text.replace('\xa0', ''))

    driver.quit()
    return paragraphs

In [ ]:
driver = webdriver.Safari()
driver.get('https://scbtechx.io/services-products/')
soup = BeautifulSoup(driver.page_source, 'html.parser')

sp_h2_content = soup.find_all('h2', class_='elementor-heading-title elementor-size-default')
# sp_links = soup.find_all('a')

sp_content = [text.text for text in sp_h2_content]

# keep_links = [link.get('href') for link in sp_links]
# filtered_list = list(filter(lambda element: element.startswith('https://scbtechx.io/'), set(keep_links)))

driver.quit()

In [ ]:
driver = webdriver.Safari()
driver.get('https://scbtechx.io/terms-of-use/')
soup = BeautifulSoup(driver.page_source, 'html.parser')

span_elements = soup.find_all('span', style='font-weight: 400;')

tou_content = [span.text.replace('\xa0', '') for span in span_elements]

driver.quit()

In [ ]:
driver = webdriver.Safari()
driver.get('https://scbtechx.io/news/')

# Define the wait duration (in seconds)
wait_duration = 5

while True:
    try:
        # Find the button element by its class
        # load_more_button = driver.find_element(By.CLASS_NAME, "wpr-load-more-btn")
        load_more_button = WebDriverWait(driver, wait_duration).until(
            EC.visibility_of_element_located((By.CLASS_NAME, "wpr-load-more-btn"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
        load_more_button.click()
        print("Clicked 'Load More' button")
        sleep(wait_duration)

    except ElementNotInteractableException:
        break

soup = BeautifulSoup(driver.page_source, 'html.parser')
# Find the section tag with the specified class
section = soup.find("section", class_="wpr-grid elementor-clearfix grid-images-loaded")

# Find all <a> tags within the section
a_tags = section.find_all("a")
keep_links = [link.get('href') for link in a_tags]
news_lisks = [link for link in set(keep_links) if link.startswith('https://scbtechx.io/') and 'https://scbtechx.io/category/' not in link]

driver.quit()

In [46]:
driver = webdriver.Safari()
driver.get('https://scbtechx.io/news/')
# load_more_button = driver.find_element(By.CLASS_NAME, "wpr-load-more-btn")

# driver.execute_script("arguments[0].scrollIntoView();", load_more_button)


load_more_button.click()


In [ ]:
links = ['https://scbtechx.io/privacy-notice/customers/',
         'https://scbtechx.io/privacy-notice/non-customers/',
         'https://scbtechx.io/data-platform/',
         'https://scbtechx.io/ekyc/',
         'https://scbtechx.io/ekyc-innovestx/']

links += [news_lisks]
pages_cons = []
for link in links:
    pages_cons.append(page_contents(link))

all_contents = pages_cons + [tou_content] + [sp_content]

In [ ]:
links = ['https://scbtechx.io/privacy-notice/customers/',
         'https://scbtechx.io/privacy-notice/non-customers/',
         'https://scbtechx.io/data-platform/',
         'https://scbtechx.io/ekyc/',
         'https://scbtechx.io/ekyc-innovestx/']

links += [news_lisks]
links

In [ ]:
all_corpus = []
for c in all_contents:
    all_corpus.append(list_of_sentence(" ".join(c)))

los_all_corpus = [list_of_sentence for corpus in all_corpus for list_of_sentence in corpus]

In [ ]:
file_path = 'src/TechX_corpus.txt'
write_list_of_lists_to_text(file_path, los_all_corpus, delimiter='\t')
my_list_of_lists = import_text_to_list_of_lists(file_path, delimiter='\t')

In [ ]:
# Check word freq
from collections import defaultdict

word_freq = defaultdict(int)
for sublist in my_list_of_lists:
    for word in sublist:
        word_freq[word] += 1
word_freq = dict(word_freq)